In [1]:
import pandas as pd
import json
import requests
import os
from dotenv import load_dotenv

load_dotenv()
API_TOKEN = str(os.getenv('API_TOKEN'))

In [2]:
def read_json_file(path: str=None, url: str=None) -> dict:
    # with open(path, 'r') as f:
    #     return json.load(f)
    response = requests.get(url)
    data = response.json()
    return data

In [3]:
path = 'https://raw.githubusercontent.com/Ciencia-de-Dados-IMT-2023/Datasets/main/spotify_million_playlist_dataset/mpd.slice.0-999.json'
data = read_json_file(url=path)

In [4]:
df_musicas = pd.DataFrame()

# Itera sobre as playlists
for playlist in data['playlists']:
    name_playlist = playlist['name']
    id_playlist = playlist['pid']

    print(f'{id_playlist} - Playlist: {name_playlist}')

    # Itera sobre as músicas da playlist
    musicas_playlist = playlist['tracks']

    for musica in musicas_playlist:
        name_musica = musica['track_name']
        id_musica = musica['track_uri']

        # print(f'\tMúsica: {name_musica} - ID: {id_musica}')
        df_musica_aux = pd.DataFrame.from_dict(musica, orient='index')
        df_musica_aux = df_musica_aux.T

        # Adiciona o nome da playlist e o id da playlist
        df_musica_aux['playlist_name'] = name_playlist
        df_musica_aux['playlist_id'] = id_playlist

        df_musica_aux = df_musica_aux.T
        df_musicas = pd.concat([df_musicas, df_musica_aux], axis=1)
        
        # break
    break

df_musicas = df_musicas.T
df_musicas.head(5)

0 - Playlist: Throwbacks


,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,playlist_name,playlist_id
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks,0
0,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks,0
0,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0
0,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,Throwbacks,0
0,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,Throwbacks,0


In [5]:
df_musicas = df_musicas[['track_uri', 'artist_uri', 'album_uri']]

df_musicas['track_uri'] = df_musicas['track_uri'].str.replace('spotify:track:', '')
df_musicas['artist_uri'] = df_musicas['artist_uri'].str.replace('spotify:artist:', '')
df_musicas['album_uri'] = df_musicas['album_uri'].str.replace('spotify:album:', '')

df_musicas.head(5)

,track_uri,artist_uri,album_uri
0,0UaMYEvWZi0ZqiDOoHU3YI,2wIVse2owClT7go1WT98tk,6vV5UrXcfyQD1wu4Qo2I9K
0,6I9VzXrHxO9rA9A5euc8Ak,26dSoYclwsYLMAKD3tpOr4,0z7pVBGOD7HCIB7S8eLkLI
0,0WqIKmW4BTrj3eJFmnCKMv,6vWDO969PvNqNYHIOW5v0m,25hVFAxTlDvXbx2X2QkUkE
0,1AWQoqb9bSvzTjaLralEkT,31TPClRtHm23RisEBtV3X7,6QPkyl04rXwTGlGlcYaRoW
0,1lzr43nnXAijIGYnCT8M8H,5EvFsr3kj42KNv97ZEnqij,6NmFmPX56pcLBOFMhIiKvF


In [91]:
# scripts/api_spotify.py

def make_req(route: str, id: str) -> requests.models.Response:
    url = f'https://api.spotify.com/v1/{route}/{id}'

    headers = {
        'Authorization': f'Bearer {API_TOKEN}'
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f'Erro: {response.status_code}')
        print(f'Erro: {response.json()}')
        import sys
        sys.exit(1)

    else:
        return response


def get_track(id_track: str) -> pd.DataFrame:
    data = make_req('tracks', id_track).json()
    df = pd.DataFrame(data['tracks'])
    return df


def get_audio_features(id_track: str) -> dict:
    data = make_req('audio-features', id_track).json()
    df = pd.DataFrame(data['audio_features'])
    return df


def get_album(id_album: str) -> dict:
    data = make_req('albums', id_album).json()
    df = pd.DataFrame(data['albums'])
    return df


def get_artist_data(id_artist: str) -> dict:
    data = make_req('artists', id_artist).json()
    df = pd.DataFrame(data['artists'])

    return df



In [108]:
def get_music_features(tracks_ids: list) -> dict:

    tracks_ids_str = ','.join(tracks_ids)
    tracks_ids_str = '?ids=' + tracks_ids_str

    # 1. Get track data
    tracks_list_df = get_track(tracks_ids_str)

    # Collect album ids
    tracks_list_df['id_album'] = tracks_list_df['album'].apply(lambda x: x['id'])

    # Collect artist ids
    tracks_list_df['id_artist'] = tracks_list_df['artists'].apply(lambda x: [d['id'] for d in x])

    # Expand artist ids to separate rows
    tracks_list_df = tracks_list_df.explode('id_artist')

    
    # 2. Get audio features
    df_audio_features = pd.DataFrame()
    for i in range(0, len(tracks_ids), 50):
        tracks_ids_str = ','.join(tracks_ids[i:i+50] )
        tracks_ids_str = '?ids=' + tracks_ids_str
        df_audio_features_aux = get_audio_features(tracks_ids_str)
        df_audio_features = pd.concat([df_audio_features, df_audio_features_aux], axis=0)

    # Filtering columns
    df_audio_features = df_audio_features[['id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]
    
    # Merge audio features with track data
    tracks_list_df = tracks_list_df.merge(df_audio_features, on='id', how='left')


    # 3. Get album data
    df_albums = pd.DataFrame()
    albums_ids = tracks_list_df['id_album'].unique().tolist()
    for i in range(0, len(albums_ids), 20):
        albums_ids_str = ','.join(albums_ids[i:i+20] )
        albums_ids_str = '?ids=' + albums_ids_str
        df_albums_aux = get_album(albums_ids_str)
        df_albums = pd.concat([df_albums, df_albums_aux], axis=0)

    # Filtering columns
    df_albums = df_albums[['id', 'name', 'popularity', 'release_date', 'total_tracks', 'type']]

    # Rename columns
    df_albums = df_albums.rename(columns={'id': 'id_album', 'name': 'album_name', 'popularity': 'album_popularity', 'release_date': 'album_release_date', 'total_tracks': 'album_total_tracks', 'type': 'album_type'})

    # Merge album data with track data
    tracks_list_df = tracks_list_df.merge(df_albums, on='id_album', how='left')
    

    # 4. Get artist data
    df_artists = pd.DataFrame()
    artists_ids = tracks_list_df['id_artist'].unique().tolist()
    for i in range(0, len(artists_ids), 50):
        artists_ids_str = ','.join(artists_ids[i:i+50] )
        artists_ids_str = '?ids=' + artists_ids_str
        df_artists_aux = get_artist_data(artists_ids_str)
        df_artists = pd.concat([df_artists, df_artists_aux], axis=0)

    # Filtering columns
    df_artists = df_artists[['id', 'name', 'genres', 'popularity', 'type', 'followers']]

    # Rename columns
    df_artists = df_artists.rename(columns={'id': 'id_artist', 'name': 'artist_name', 'genres': 'artist_genres', 'popularity': 'artist_popularity', 'type': 'artist_type', 'followers': 'artist_followers'})

    # Merge artist data with track data
    tracks_list_df = tracks_list_df.merge(df_artists, on='id_artist', how='left')

    # Collect number os followers
    tracks_list_df['artist_followers'] = tracks_list_df['artist_followers'].apply(lambda x: x['total'])

    # Expand artist ids to separate rows
    tracks_list_df = tracks_list_df.explode('artist_genres')

    # Keep only relevant artist_genres
    elevant_artist_genres = ['pop', 'hip hop', 'r&b', 'rap', 'reggae', 'rock', 'punk', 'alternative']
    tracks_list_df = tracks_list_df[tracks_list_df['artist_genres'].isin(elevant_artist_genres)]

    tracks_list_df = tracks_list_df[[
        'id', 'id_album', 'id_artist', 'name', 'explicit', 'duration_ms', 'popularity', # Track data
        'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence', # Audio features
        'album_name', 'album_popularity', 'album_release_date', 'album_total_tracks', 'album_type', # Album data
        'artist_name', 'artist_genres', 'artist_popularity', 'artist_type', 'artist_followers' # Artist data
        ]]

    return tracks_list_df


In [118]:
tracks_ids = df_musicas['track_uri'].unique().tolist()

In [119]:
df_musics = pd.DataFrame()
for i in range(0, len(tracks_ids), 50):
    df_aux = get_music_features(tracks_ids[i:i+50])
    df_musics = pd.concat([df_musics, df_aux], axis=0)

In [120]:
df_musics.iloc[0]

id                                        0UaMYEvWZi0ZqiDOoHU3YI
id_album                                  6vV5UrXcfyQD1wu4Qo2I9K
id_artist                                 2wIVse2owClT7go1WT98tk
name                  Lose Control (feat. Ciara & Fat Man Scoop)
explicit                                                    True
duration_ms                                               226863
popularity                                                    69
acousticness                                              0.0311
danceability                                               0.904
energy                                                     0.813
instrumentalness                                         0.00697
key                                                            4
liveness                                                  0.0471
loudness                                                  -7.105
mode                                                           0
speechiness              

In [114]:
df_musics['id'].nunique()

38